<div class="alert alert-block alert-success">
<b>COMENTARIO GENERAL CUARTA REVISIÓN</b> <a class="tocSkip"></a>

Sara, felicidades! Tu trabajo está aprobado. No tengo más que decir, aplicaste las correcciones impecablemente. Sigue esforzándote así! :)

</div>

# Proyecto SQL: Aplicaciones para amantes de los libros

# Contents

# 1. Introducción y Objetivos 
* [Introducción, objetivos y descripción de los datos](#intro)

# 2. Exploración

* [Muestra de las tablas](#intro)

# 3. Consultas 
* [Encuentra el número de libros publicados después del 1 de enero de 2000](#c1)
* [Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro](#c2)
* [Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis)](#c3)
* [Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.](#c4)
* [Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.](#c5)


# 4. Conclusión 

* [Conclusiones y Recomendaciones](#conclusion) 

# 1. Introducción, objetivos y descripción de los datos

El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

Tenemos una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros.

El objetivo es generar una propuesta de valor para un nuevo producto.

### Descripción de los datos

`**books**`

Contiene datos sobre libros:

- `book_id`: identificación del libro
- `author_id`: identificación del autor o autora
- `title`: título
- `num_pages`: número de páginas
- `publication_date`: fecha de la publicación
- `publisher_id`: identificación de la editorial

`**authors**`

Contiene datos sobre autores:

- `author_id`: identificación del autor o autora
- `author`: el autor o la autora

`**publishers**`

Contiene datos sobre editoriales:

- `publisher_id`: identificación de la editorial
- `publisher`: la editorial

`**ratings**`

Contiene datos sobre las calificaciones de usuarios:

- `rating_id`: identificación de la calificación
- `book_id`: identificación del libro
- `username`: el nombre del usuario que revisó el libro
- `rating`: calificación

`**reviews**`

Contiene datos sobre las reseñas de los y las clientes:

- `review_id`: identificación de la reseña
- `book_id`: identificación del libro
- `username`: el nombre del usuario que revisó el libro
- `text`: el texto de la reseña

Primero realizaremos la importación de las librerías y la conexión con la fuente de los datos.

In [1]:
# importar librerías y conexión de los datos 
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
        db_config['user'],
        db_config['pwd'],
        db_config['host'],
        db_config['port'],
        db_config['db'],
    )


engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# 2. Exploración 

Estudiaremos los datos de los 5 archivos y mostraremos las primeras filas. El objetivo es conocer los campos que comparten en comun entre los dataset que nos servirán para conocer el diagrama de datos.


In [2]:
# Primer dataset: Books
books = ''' SELECT * 
                FROM books;
            '''

In [3]:
pd.io.sql.read_sql(books, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [4]:
unique_books_id = ''' SELECT COUNT(DISTINCT book_id) 
                FROM books;
            '''

pd.io.sql.read_sql(unique_books_id, con = engine)

,count
0,1000


In [5]:
unique_books_name = ''' SELECT COUNT(DISTINCT title) 
                FROM books;
            '''

pd.io.sql.read_sql(unique_books_name, con = engine)

,count
0,999


* Tenemos 1000 registros de datos de los libros, id del autor, autor, numero de paginas, titulo y fecha de la publicación.
* La clave es book _id el cual tiene valores unicos por cada libro y nos coincide.
* Sin embargo tenemos 999 nombres de libros unicos, es decir que hay un libro que esta repetido el nombre dos veces pero con diferente id.

In [6]:
unique_author_id = ''' SELECT COUNT(DISTINCT author_id) 
                FROM books;
            '''

pd.io.sql.read_sql(unique_author_id, con = engine)

,count
0,636


* Existen 636 autores unicos dentro de los 1000 registros, por lo que varios de ellos debieron haber escrito 1 o mas libors. 

In [7]:
unique_publisher_id = ''' SELECT COUNT(DISTINCT publisher_id) 
                FROM books;
            '''

pd.io.sql.read_sql(unique_publisher_id, con = engine)

,count
0,340


* Existen 340 editoriales unicas que publicaron los 1000 libros. Esto tambien nos informa que hay editoriales que publicaron mas de 1 libro. 

In [8]:
# Segundo dataset: authors 
authors= '''SELECT * 
            FROM authors;
         '''
pd.io.sql.read_sql(authors, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


* En este dataset encontramos unicamente el nombre del autor con su id correspondiente. 
* Encontramos en común la columna author_id con respecto al dataset anterior, y son los mismos 636 numeros de autores reflejados en este dataset y el de books.

In [9]:
# Tercer dataset: publishers 
publishers= ''' SELECT *
                FROM publishers;
            '''
pd.io.sql.read_sql(publishers, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


* Encontramos que existen 340 editoriales en el archivo y que coinciden con el dataset de books.
* La columna publisher id tiene en comun con el dataset de books.

In [10]:
#Cuarto dataset: ratings
ratings= '''SELECT * 
            FROM ratings;
         '''
pd.io.sql.read_sql(ratings, con=engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [11]:
# Libros unicos con calificacion
libros_unicos_ratings= '''SELECT COUNT(DISTINCT book_id)
                            FROM ratings;
                         '''
pd.io.sql.read_sql(libros_unicos_ratings, con=engine)

,count
0,1000


In [12]:
# Usuarios unicos que calificaron 
usuarios_unicos_ratings= '''SELECT COUNT(DISTINCT username)
                            FROM ratings;
                         '''
pd.io.sql.read_sql(usuarios_unicos_ratings, con=engine)

,count
0,160


* El archivo contiene 6,456 calificaciones de los usuarios que revisaron los libros. 
* La columna book_id se encuentra tambien en el primer dataset de books.
* Existen 160 usuarios unicos que han calificado a los libros, es decir que mas un usuario pudo haber calificado 1 o mas libros.

In [13]:
# Quinto data set: reviews
reviews= '''SELECT * 
            FROM reviews;
         '''
pd.io.sql.read_sql(reviews, con=engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


* Obtuvimos 2,793 reseñas, pero puede que no todos los libros hayan recibido la reseña, asi que haremos la siguiente consulta a continuación.

In [14]:
#Cuantos libros unicos tuvieron el review 

unicos_libros_review= ''' SELECT COUNT(DISTINCT book_id) AS libros_unicos
                          FROM reviews;
                          '''
pd.io.sql.read_sql(unicos_libros_review,con=engine)

,libros_unicos
0,994


In [15]:
# Usuarios unicos que calificaron 
usuarios_unicos_reviews= '''SELECT COUNT(DISTINCT username)
                            FROM reviews;
                         '''
pd.io.sql.read_sql(usuarios_unicos_reviews, con=engine)

,count
0,160


* Podemos ver que 6 libros no tuvieron la reseña, pero para la mayoría  de libros si disponemos esa información.
* Los 160 usuarios que dejaron la calificacion tambien dejaron la reseña.

**Conclusiones intermedias**

* Tenemos 1000 libros unicos bajo el id.
* Hay 2 registros de libros que tiene repetido el mismo titulo.
* Existen 636 autores unicos
* Hay 340 editoriales unicos
* 1000 libros tuvieron una calificacion 
* 994 libros tuvieron una reseña
* En total hubieron 2,793 reseñas
* 160 usuarios participaron en dar el review y tambien la calificación.

<div class="alert alert-block alert-success">
<b>Éxito</b> <a class="tocSkip"></a>

Genial que realizaras todas estas revisiones, excelente Sara. Poc@s estudiantes lo hacen</div>

# 3. Consultas 

**3.1 Encuentra el número de libros publicados después del 1 de enero de 2000.**

In [16]:
#Books
numero_libros= '''SELECT COUNT (book_id) AS numero_de_libros
                  FROM books
                  WHERE publication_date > '2000-01-01';
               '''
pd.io.sql.read_sql(numero_libros, con=engine)

,numero_de_libros
0,819


* Tenemos 819 libros publicados despues del 1 de enero del 2000. 

<div class="alert alert-block alert-success">
<b>Éxito</b> <a class="tocSkip"></a>

En efecto</div>

**3.2 Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.**

In [17]:
# Books, Reviews, Rating 
consulta2 = '''SELECT books.title AS libro, COUNT(DISTINCT reviews.review_id)AS n_reviews, AVG(ratings.rating) AS promedio
           FROM books 
           LEFT JOIN reviews ON reviews.book_id=books.book_id
           LEFT JOIN ratings ON ratings.book_id = books.book_id
           
           GROUP BY libro
           ORDER BY n_reviews DESC;

'''

pd.io.sql.read_sql(consulta2, con =engine)

,libro,n_reviews,promedio
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Glass Castle,6,4.206897
4,Eat Pray Love,6,3.395833
...,...,...,...
994,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
995,The Natural Way to Draw,0,3.000000
996,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
997,Essential Tales and Poems,0,4.000000


* Al hacer el primer left join de books con las reseñas nos quedamos con valores ausentes en los 6 libros que no recibieron reseñas, es por eso que les reemplazamos con 0. 
* Ordenamos el resultado para ver cuales eran los libros mas populares que recibieron mas número de reseñas, siendo el maximo 7 y con una puntuacion de 3.66

**3.3 Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).**

In [18]:
#Books, publishers
consulta3= '''
              SELECT COUNT(books.book_id) AS numero_de_libros,publishers.publisher
FROM books INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE num_pages > 50 
GROUP BY publishers.publisher
ORDER BY numero_de_libros DESC
LIMIT 1
              ;
              
            '''
pd.io.sql.read_sql(consulta3, con=engine)

,numero_de_libros,publisher
0,42,Penguin Books


* La editorial que ha publicado mas libros con mas de 50 paginas es Penguin Books. 


**3.4 Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.**

In [19]:
#Books,Author,Ratings 
consulta4= '''     
            SELECT authors.author, AVG(ratings.rating) AS promedio
            FROM books LEFT JOIN authors ON books.author_id = authors.author_id
            LEFT JOIN ratings ON books.book_id = ratings.book_id
            WHERE books.title IN (SELECT books.title
               FROM books INNER JOIN ratings ON books.book_id= ratings.book_id
               GROUP BY books.title
               HAVING COUNT(ratings.rating) >49)
            GROUP BY authors.author
            ORDER BY promedio DESC
            LIMIT 1
              ;
              
            '''
pd.io.sql.read_sql(consulta4, con=engine)

,author,promedio
0,J.K. Rowling/Mary GrandPré,4.287097


* El autor es J.K. Rowling/Mary GrandPré que recibio calificacion promedio de 4.28.

**3.5 Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.**


In [20]:
#Ratings, Reviews

subconsulta5= '''
            SELECT AVG(SUB.n_review)
            FROM 
                (SELECT COUNT(review_id) AS n_review,username
                FROM reviews
                WHERE username IN (SELECT username
                FROM ratings
                GROUP BY username
                HAVING COUNT( rating_id) >50
                  )
                GROUP BY username    
             
              ) AS SUB
              ;
              
            '''
pd.io.sql.read_sql(subconsulta5, con=engine)

,avg
0,24.333333


* El numero promedio de reseñas fue de 24. 

# 4. Conclusiones

* La propuesta de valor es que se pueda proporcionar un sistema de recomendaciones de libros para que se puedan compartir entre usuarios y tener un sitio web donde tengamos un foro para que puedan intercambiar opiniones, reseñas, recomendar autores,etc. 

* Esto permitira que se pueda aumentar la cobertura de usuarios actualmente que tuvimos de 160 usuarios, la idea es que se genere valor a traves de ellos mismos con los que puedan aportar e incentivar la lectura.

* Adicionalmente dado la situcion de la pandemia, se tendria un mayor engagement si se realizan clubes de lectura en vivo y mantener el interes y el compromiso de los usuarios.

* Establecer un convenio y alianzas con la editorial Penguin Books quien fue el que mayor numero de libros se han publicado con mas de 50 paginas, esto nos permitirá tener variedad de libros en nuestro catalogo de producto.